In [1]:
# The code has two aims: 
# 1. To clean Chinese text of the city-level information and
# 2. To merge city-level scores of all kinds with our data

import pandas as pd
import numpy as np
import random
from functools import reduce


# 1.1 Read the data and unify the name for four 直辖市 and some 省直管 districts

city = pd.read_excel("cities.xlsx").drop("Citycode", axis=1).rename(columns={"Cityname": "city"})
data = pd.read_csv("output.csv")

pc = {50:"重庆", 11:"北京", 12:"天津", 31:"上海"}

for key, value in pc.items():
    data.loc[data["Destination"] == key, "city"] = value
    
data.loc[(data['city'] == "省直辖") & (data['Destination'] == 42), "city"] = "湖北省直管"

    
# 1.2 Unify city names in the data file with those in the city file, enabling double-ended check 

def modify_row(row, compare_data):     
    for ind in compare_data['city']:
        if (ind.find(row) != -1) | (row.find(ind) != -1):
            row = ind
            break
    return row  

data['city'] = data['city'].apply(lambda row: modify_row(row, city))



# 1.3.1 First round check: Discover the city names in data file that have not been matched

def merge_check(data, city):
    ud = pd.DataFrame(data['city'].unique()).rename(columns={0: "c1"})
    uc = pd.DataFrame(city['city'].unique()).rename(columns={0: "c2"})
    data_city = pd.merge(ud, uc, how ='left', left_on = "c1", right_on = "c2")
    return data_city[data_city['c2'].isnull() == True]

check_1st = merge_check(data, city) 
len(check_1st)                        # Total 28 names have not been matched yet



# Manually inspect them one by one and document these names in a dictionary

'''for i in data['city'].unique():
    if i.find("红河") != -1: # "黔南"
        print(i)'''


d = {"阿坝藏":"阿坝州", "博州":"博尔塔拉州", "博尔塔拉蒙古自治州":"博尔塔拉州", "海西州":"海西蒙古族藏族自治州", 
     "黔东南州":"黔东南苗族侗族自治州", "黔东南州苗族侗族自治":"黔东南苗族侗族自治州", "海南州":"海南藏族自治州", 
     "红河州哈尼":"红河哈尼族彝族自治州", "红河洲":"红河哈尼族彝族自治州", "红河州":"红河哈尼族彝族自治州", 
     "海东市":"海东地区", "昌吉回族自":"昌吉州", "昌吉回族自治州":"昌吉州", "湘西土家族":"湘西州", "湘西土家族苗族自治州":"湘西州",
     "伊犁州":"伊犁哈萨克自治州",  "日喀则市":"日喀则地区", "黔南州":"黔南布依族苗族自治州",  "吐鲁番市":"吐鲁番地区",
     "甘孜藏族自治州":"甘孜州", "巴音郭楞蒙古族自治州":"巴音郭楞蒙古自治州", "西双版纳傣族自治州":"西双版纳州",
     "黔西南布依族苗族自治":"黔西南州", "海北藏族自治州":"海北州", "甘南藏族自治州": "甘南州"}



# 1.3.2 Second round check

for old, new in d.items():
    data.loc[data["city"] == old, "city"] = new
    
check_2nd = merge_check(data, city)
len(check_2nd)                        # All names have been matched now :)


# 1.4 Randomly generage a series of pseudo city code

random.seed(7735)
code = random.sample(range(1,len(data['city'].unique())+1), len(data['city'].unique()))
citycode = pd.DataFrame({"city":data['city'].unique(), "citycode":code})
city = pd.merge(city, citycode, left_on = "city", right_on = "city")

In [24]:
# 2. Merge city-level scores

# 2.1 Classifying city size based on population and merge three files from above data, city & citycode

city["city_size"] = city["2013 Population (10000 persons)"]

def resize(value):
    if value < 50:
        value = 11
    elif value < 100:
        value = 12
    elif value < 300:
        value = 13
    elif value < 500:
        value = 14
    else:
        value = 15
    return value
        
city["city_size"] = city["city_size"].apply(lambda row: resize(row))
# city["city_size"].value_counts() 

data_city = pd.merge(data, city, how='left', left_on = "city", right_on = "city")

# 2.2 Reshape the file containing scores info for merging purpose

scores = pd.read_excel(r"c:\Users\User\Desktop\Clustering\Scores_City.xlsx", index_col = 0)
s = scores.iloc[:, :24]
m = scores.iloc[:, 24:48]
b1 = scores.iloc[:, 48:72]
b2 = scores.iloc[:, 72:96]
mega = scores.iloc[:, 96:120]

scores = pd.DataFrame() 

for (df, size) in zip([s, m, b1, b2, mega], [11, 12, 13, 14, 15]):
    df["_size"] = size                                                    # create the city size variable for each df
    df.columns = ["city_" + col[col.find("_")+1: ] for col in df.columns] # rename the column to be the same across city sizes
    scores = scores.append(df)                                            # append all the dfs together

# 2.3 
scores = scores.reset_index()
output = pd.merge(data_city, scores, how = "left", left_on = ["Destination", "city_size"], right_on = ["Procode", "city_size"])

In [29]:
output.to_csv("../3. SelectionAnalysis/output_city.csv", encoding = "utf8") #output.to_excel("output_city.xlsx")